In [1]:
from pymatgen import Structure, Lattice, PeriodicSite
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

In [2]:
# Define the lattice parameter, in Å
lattice_parameter = 5.43 # in A

# Create a cubic lattice with the lattice parameter above
cubic_lattice = Lattice.cubic(lattice_parameter)

# Create a list of coordinates (i.e. [[x, y, z], [x2, y2, z2], ...])
DC_coordinates = [ [0, 0, 0], [0.25, 0.25, 0.25], [0.5, 0.5, 0], [0.5, 0, 0.5], [0, 0.5, 0.5], [0.25, 0.75, 0.75], [0.75, 0.25, 0.75], [0.75, 0.75, 0.25]] 

DC_species = ["Si" for coord in DC_coordinates] # creates the list of Si species used when creating the Structure

# Create the structure
Si_DC = Structure(cubic_lattice, species=DC_species, coords=DC_coordinates ) # creates the Structure

# Simplify the structure to a primitive one
Si_DC = SpacegroupAnalyzer(Si_DC).get_primitive_standard_structure()

In [3]:
print(Si_DC)

Full Formula (Si2)
Reduced Formula: Si
abc   :   3.839590   3.839590   3.839590
angles:  60.000000  60.000000  60.000000
Sites (2)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  Si    0     0     0
  1  Si    0.25  0.25  0.25


In [4]:
from pymatgen.io.vasp.inputs import Kpoints, Poscar, Potcar, Incar

In [5]:
POSCAR = Poscar(Si_DC)
print(POSCAR)

Si2
1.0
0.000000 2.715000 2.715000
2.715000 0.000000 2.715000
2.715000 2.715000 0.000000
Si
2
direct
0.000000 0.000000 0.000000 Si
0.250000 0.250000 0.250000 Si



In [6]:
# fill in this tuple with desired grid dimensions (e.g. (a, b, c))
grid = (8, 8, 8)
# We're using the monkhorst_automatic constructor here to create a grid via the Monkhorst-Pack method.
KPOINTS = Kpoints.monkhorst_automatic(kpts=grid)
print(KPOINTS)
print(POSCAR)

Automatic kpoint scheme
0
Monkhorst
8 8 8

Si2
1.0
0.000000 2.715000 2.715000
2.715000 0.000000 2.715000
2.715000 2.715000 0.000000
Si
2
direct
0.000000 0.000000 0.000000 Si
0.250000 0.250000 0.250000 Si



In [7]:
my_incar_params = {"EDIFF" : 0.0004, 
                "EDIFFG" : -0.01, 
                "ENCUT" : 350, 
                "IBRION" : 2, 
                "ISIF" : 3, 
                "ISMEAR" : 0, 
                "SIGMA" : 0.2, 
                "NSW" : 15,
                "NBANDS" : 8,
                "LREAL" : False}

In [8]:
INCAR = Incar.from_dict(my_incar_params)
print(INCAR)

EDIFF = 0.0004
EDIFFG = -0.01
ENCUT = 350
IBRION = 2
ISIF = 3
ISMEAR = 0
LREAL = False
NBANDS = 8
NSW = 15
SIGMA = 0.2



In [9]:
def write_input_set(POTCAR, POSCAR, INCAR, KPOINTS, directory="new_input_set", use_fake_potcar=True):
    
    import os
    if not os.path.exists(directory):
        os.makedirs(directory)

    if not use_fake_potcar:
        POTCAR.write_file("{}/POTCAR".format(directory))
    else:
        from shutil import copyfile
        # Copy over a fake potcar
        copyfile("fake_vasp_data/3kvi0H85jC/POTCAR", "directory")
    
    POSCAR.write_file("{}/POSCAR".format(directory))
    INCAR.write_file("{}/INCAR".format(directory))
    KPOINTS.write_file("{}/KPOINTS".format(directory))
    print("Input set written to '{}'".format(directory))

In [10]:
# Directory where you want to save the files. e.g. "my_part_1/Si_DC"
my_directory = "my_part_1/Si_DC"
write_input_set(None, POSCAR, INCAR, KPOINTS, directory=my_directory)

Input set written to 'my_part_1/Si_DC'


In [11]:
!python fake_vasp.py "my_part_1/Si_DC"

 running on    8 total cores
 distrk:  each k-point on    8 cores,    1 groups
 distr:  one band on    1 cores,    8 groups
 using from now: INCAR     
 vasp.5.4.4.18Apr17-6-g9f103f2a35 (build Dec 09 2018 21:25:21) complex          
  
 POSCAR found type information on POSCAR  Si
 POSCAR found :  1 types and       2 ions
 scaLAPACK will be used

 ----------------------------------------------------------------------------- 
|                                                                             |
|           W    W    AA    RRRRR   N    N  II  N    N   GGGG   !!!           |
|           W    W   A  A   R    R  NN   N  II  NN   N  G    G  !!!           |
|           W    W  A    A  R    R  N N  N  II  N N  N  G       !!!           |
|           W WW W  AAAAAA  RRRRR   N  N N  II  N  N N  G  GGG   !            |
|           WW  WW  A    A  R   R   N   NN  II  N   NN  G    G                |
|           W    W  A    A  R    R  N    N  II  N    N   GGGG   !!!           |
|           

In [12]:
from pymatgen.io.vasp.outputs import Outcar
OUTCAR = Outcar(my_directory + "/OUTCAR")
OUTCAR.final_energy

-10.84749086

In [13]:
## Beta Tin Si Structure 
# Define the lattice parameters, in Å
a = 4.9
c = 2.548

# Create the tetragonal lattice
# Create the tetragonal lattice
tetragonal_lattice = Lattice([(a, 0, 0), (0, a, 0), ( a/2, a/2, c/2)])
#tetragonal_lattice = Lattice.tetragonal(lattice_parameter_a, lattice_parameter_c)

# Create a list of coordinates (i.e. [ [x1, y1, z1], [x2, y2, z2], ... ])
BSn_coordinates = [[-0.125, -0.375, 0.25, ], [0.125, 0.375, -0.25]]
BSn_species = ["Si" for coord in BSn_coordinates] # creates the list of Si species used when creating the Structure

Si_BSn = Structure(tetragonal_lattice, species=BSn_species, coords=BSn_coordinates ) # creates the Structure

In [14]:
my_directory = "my_part_1/Si_BSn"

POSCAR = Poscar(Si_BSn)
print(POSCAR)
write_input_set(None, POSCAR, INCAR, KPOINTS, directory=my_directory)

Si2
1.0
4.900000 0.000000 0.000000
0.000000 4.900000 0.000000
2.450000 2.450000 1.274000
Si
2
direct
-0.125000 -0.375000 0.250000 Si
0.125000 0.375000 -0.250000 Si

Input set written to 'my_part_1/Si_BSn'


In [15]:
!python fake_vasp.py "my_part_1/Si_BSn"

 running on    8 total cores
 distrk:  each k-point on    8 cores,    1 groups
 distr:  one band on    1 cores,    8 groups
 using from now: INCAR     
 vasp.5.4.4.18Apr17-6-g9f103f2a35 (build Dec 09 2018 21:25:21) complex          
  
 POSCAR found type information on POSCAR  Si
 POSCAR found :  1 types and       2 ions
 scaLAPACK will be used

 ----------------------------------------------------------------------------- 
|                                                                             |
|           W    W    AA    RRRRR   N    N  II  N    N   GGGG   !!!           |
|           W    W   A  A   R    R  NN   N  II  NN   N  G    G  !!!           |
|           W    W  A    A  R    R  N N  N  II  N N  N  G       !!!           |
|           W WW W  AAAAAA  RRRRR   N  N N  II  N  N N  G  GGG   !            |
|           WW  WW  A    A  R   R   N   NN  II  N   NN  G    G                |
|           W    W  A    A  R    R  N    N  II  N    N   GGGG   !!!           |
|           

 opt :   1.8927  next Energy=   -10.272284 (dE=-0.630E-01)
 bond charge predicted
       N       E                     dE             d eps       ncg     rms          rms(c)
DAV:   1    -0.102722729670E+02   -0.42817E-07   -0.33953E-09   448   0.365E-04    0.456E-05
DAV:   2    -0.102722729662E+02    0.82616E-09   -0.21686E-10   448   0.116E-04
  11 F= -.10272273E+02 E0= -.10263035E+02  d E =-.630339E-01
 curvature:  -0.00 expect dE=-0.594E-07 dE for cont linesearch -0.587E-08
 ZBRENT: interpolating
 opt :   1.8926  next Energy=   -10.272284 (dE=-0.630E-01)
 bond charge predicted
       N       E                     dE             d eps       ncg     rms          rms(c)
DAV:   1    -0.102722729820E+02   -0.15020E-07   -0.40977E-10   448   0.127E-04    0.163E-05
DAV:   2    -0.102722729825E+02   -0.46850E-09   -0.25165E-11   448   0.394E-05
  12 F= -.10272273E+02 E0= -.10263035E+02  d E =-.630339E-01
 curvature:  -0.00 expect dE=-0.210E-07 dE for cont linesearch -0.207E-08
 ZBRENT: inte

In [16]:
from pymatgen.io.vasp.outputs import Outcar
OUTCAR = Outcar(my_directory + "/OUTCAR")
OUTCAR.final_energy

-10.27227299

In [17]:

def make_convergence_calc(structure, ENCUT, n_kpoints, output_dir, incar_params, use_fake_potcar=True):
    if not use_fake_potcar:
        # make POTCAR
        POTCAR = Potcar(["Si"], functional="PBE")
    else:
        POTCAR = None
    
    # make POSCAR from primitive unit cell of structure
    POSCAR = Poscar(SpacegroupAnalyzer(structure).get_primitive_standard_structure())
    
    # make KPOINTS from specified number of kpoints (n,n,n)
    grid = (n_kpoints, n_kpoints, n_kpoints)
    KPOINTS = Kpoints.monkhorst_automatic(kpts=grid)
    
    # Use our previous incar settings as starting point
    incar_params = dict(incar_params) # make a copy so we don't mutate original dict
    incar_params["ENCUT"] = ENCUT
    INCAR = Incar.from_dict(incar_params)
    
    # Write our input set to the directory
    write_input_set(POTCAR, POSCAR, INCAR, KPOINTS, directory=output_dir)
    
    
import matplotlib.pyplot as plt
from math import log

# Helper function to plot convergence test data.
def plot_convergence(parameters, energies, xlabel="Cutoff Energy, $E_{cut} $ (eV)", 
                     ylabel="Final Energy (eV/atom)", title="Final Energy vs Cutoff Energy",
                     save_fig=True, filename="B-Sn_KPoints_conv.png"):
    """Helper function to create convergence plots. 
    
    Arguments:
        parameters (list of int/float): independent variable (e.g. ENCUT or KPOINTS)
        energies (list of float): resulting final energies
        xlabel (str): label for the x-axis
        ylabel (str): label for you y-axis
        title (str): title for the plot
        save_fig (bool): if True, figure is saved as a picture
        filename (str): filename of picture
        """
    
    # plot the data
    plt.plot(parameters, energies, 'rx')
    plt.plot(parameters, energies, 'b-')
    
    # add labels to the plot
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    
    # saves the plot in convergence_test_results/plots/ (WILL OVERWRITE FILES IF FILENAME IS NOT CHANGED)
    if save_fig:
        plt.savefig("convergence_tests/plots/B-Sn_KPoints_conv.png", bbox_inches='tight'
        #plt.savefig("convergence_tests/plots/B_Sn_KPoints_conv.png", bbox_inches='tight')
    # show the plot here
    plt.show()

SyntaxError: invalid syntax (<ipython-input-17-bdc95a741fb0>, line 58)

In [ ]:

example_ENCUTS = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
example_energies = [-12.33213554, -10.15439674, -10.34140392, -10.25157338, -10.27502587, -10.29029755, -10.24991292, -10.25142488, -10.24409763, -10.24372507, -10.25303384]


# Example usage for plot_convergence
plot_convergence(example_ENCUTS, example_energies, 
                 xlabel="Cutoff K point", 
                 ylabel="Final Energy (eV/atom)", 
                 title = "Final Energy vs Cutoff K Point for B-Sn Si")

In [ ]:
# list of ENCUT values (e.g. [ENCUT_1, ENCUT_2, ...])
Ecs = [150, 200, 250, 300, 350]

# writes the ENCUT convergence test input sets to convergence_tests/B_Sn/
for E in Ecs:
    dirname = "convergence_tests/DC/E_{}_K_{}".format(E, 6)
    # We're using a (6,6,6) k-points grid for this convergence test
    make_convergence_calc(Si_DC, E, 6, output_dir=dirname, incar_params=my_incar_params)

In [ ]:
# ENCUT value determined from last convergence test
ENCUT = [250]

# list of KPOINTS densities (n, n, n) (e.g. [k1, k2,, k3, ...])
Ks = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

# write the KPOINTS convergence test input sets to convergence_tests/DC/
for K in Ks:
    dirname = "convergence_tests/DC/E_{}_K_{}".format(ENCUT, K)
    make_convergence_calc(Si_DC, ENCUT, K, output_dir=dirname, incar_params=my_incar_params)